# 🚀 Multi-Model Prompt Engineering Testing Notebook

This notebook tests your complete AI setup with **OpenAI**, **Llama**, and **Google Gemini**.

## 🎯 What This Notebook Does:
- ✅ Verifies all API connections
- 🔄 Compares responses across models
- 🧪 Runs prompt engineering experiments
- 📊 Provides interactive testing environment

---

## 1️⃣ Environment Setup and Verification

In [ ]:
# Import required libraries and load environment
import os
import sys

# Install required packages if missing
required_packages = ['python-dotenv', 'openai', 'google-generativeai', 'requests']

for package in required_packages:
    try:
        if package == 'python-dotenv':
            import dotenv
        elif package == 'google-generativeai':
            import google.generativeai as genai
        else:
            __import__(package)
        print(f"✅ {package} is available")
    except ImportError:
        print(f"📦 Installing {package}...")
        import subprocess
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
        print(f"✅ {package} installed")

print("\n🎉 All required packages are ready!")

In [ ]:
# Load environment variables
from dotenv import load_dotenv
import requests
import json

# Try to load .env from multiple locations
env_loaded = False
for env_path in ['.env', '../.env', '/Users/rosalinatorres/Documents/.env']:
    if os.path.exists(env_path):
        load_dotenv(env_path)
        print(f"✅ Loaded environment from: {env_path}")
        env_loaded = True
        break

if not env_loaded:
    print("❌ .env file not found")
    print("Please ensure your .env file is in the right location")
else:
    # Check API keys
    api_keys = {
        'OPENAI_API_KEY': os.getenv('OPENAI_API_KEY'),
        'LLAMA_API_KEY': os.getenv('LLAMA_API_KEY'),
        'GOOGLE_API_KEY': os.getenv('GOOGLE_API_KEY'),
        'GOOGLE_API_KEY_BACKUP': os.getenv('GOOGLE_API_KEY_BACKUP')
    }
    
    print("\n🔐 API Key Status:")
    for key_name, key_value in api_keys.items():
        if key_value and key_value != "your-openai-key-here":
            masked_key = f"{key_value[:8]}...{key_value[-8:]}"
            print(f"✅ {key_name}: {masked_key}")
        else:
            print(f"❌ {key_name}: Not configured")

## 2️⃣ Initialize Multi-Model Platform

In [ ]:
class NotebookMultiModelPlatform:
    """Simplified multi-model platform for Jupyter notebooks"""
    
    def __init__(self):
        self.openai_key = os.getenv('OPENAI_API_KEY')
        self.llama_key = os.getenv('LLAMA_API_KEY')
        self.google_key = os.getenv('GOOGLE_API_KEY')
        self.google_backup_key = os.getenv('GOOGLE_API_KEY_BACKUP')
        
        self.clients = {}
        self.setup_clients()
    
    def setup_clients(self):
        """Initialize API clients"""
        print("🔧 Setting up API clients...")
        
        # OpenAI
        if self.openai_key and self.openai_key != "your-openai-key-here":
            try:
                from openai import OpenAI
                self.clients['openai'] = OpenAI(api_key=self.openai_key)
                print("✅ OpenAI client ready")
            except Exception as e:
                print(f"❌ OpenAI setup failed: {e}")
        else:
            print("⚠️  OpenAI key not configured")
        
        # Google Gemini
        google_keys = [self.google_key, self.google_backup_key]
        google_keys = [k for k in google_keys if k]  # Remove None values
        
        for i, key in enumerate(google_keys):
            try:
                import google.generativeai as genai
                genai.configure(api_key=key)
                model = genai.GenerativeModel('gemini-pro')
                
                # Test with minimal request
                test_response = model.generate_content(
                    "Hi", generation_config={'max_output_tokens': 5}
                )
                
                self.clients['google'] = model
                key_type = "Primary" if i == 0 else "Backup"
                print(f"✅ Google Gemini ready ({key_type} key)")
                break
                
            except Exception as e:
                key_type = "Primary" if i == 0 else "Backup"
                print(f"❌ Google {key_type} key failed: {str(e)[:50]}")
                continue
        
        if self.llama_key:
            print("✅ Llama key available (HTTP-based)")
        else:
            print("⚠️  Llama key not configured")
    
    def test_openai(self, prompt):
        """Test OpenAI API"""
        if 'openai' not in self.clients:
            return "❌ OpenAI not available"
        
        try:
            response = self.clients['openai'].chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"❌ OpenAI Error: {str(e)[:100]}"
    
    def test_llama(self, prompt):
        """Test Llama API"""
        if not self.llama_key:
            return "❌ Llama not available"
        
        try:
            url = "https://api.llama.com/v1/chat/completions"
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {self.llama_key}"
            }
            data = {
                "model": "Llama-4-Maverick-17B-128E-Instruct-FP8",
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": 100
            }
            
            response = requests.post(url, headers=headers, json=data, timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                return result['choices'][0]['message']['content']
            else:
                return f"❌ Llama API Error: {response.status_code}"
        except Exception as e:
            return f"❌ Llama Error: {str(e)[:100]}"
    
    def test_google(self, prompt):
        """Test Google Gemini API"""
        if 'google' not in self.clients:
            return "❌ Google not available"
        
        try:
            response = self.clients['google'].generate_content(
                prompt,
                generation_config={'max_output_tokens': 100}
            )
            return response.text
        except Exception as e:
            return f"❌ Google Error: {str(e)[:100]}"
    
    def compare_models(self, prompt, title="Model Comparison"):
        """Compare responses across all models"""
        print(f"\n🎯 {title}")
        print(f"Prompt: {prompt}")
        print("=" * 60)
        
        models = {
            "OpenAI GPT-3.5": self.test_openai,
            "Meta Llama": self.test_llama,
            "Google Gemini": self.test_google
        }
        
        results = {}
        for model_name, test_func in models.items():
            print(f"\n🤖 {model_name}:")
            print("-" * 30)
            response = test_func(prompt)
            print(response)
            results[model_name] = response
        
        return results

# Initialize the platform
platform = NotebookMultiModelPlatform()
print("\n🎉 Multi-Model Platform Ready!")

## 3️⃣ Connection Tests

In [ ]:
# Quick connection test
test_prompt = "Hello! Respond with just 'API working!' to test the connection."
results = platform.compare_models(test_prompt, "🧪 Connection Test")

# Count working services
working_services = [name for name, response in results.items() if not response.startswith("❌")]
print(f"\n📊 Summary: {len(working_services)}/3 services are working")

if len(working_services) > 0:
    print("✅ You're ready for prompt engineering experiments!")
else:
    print("❌ Please check your API keys and internet connection")

## 4️⃣ Prompt Engineering Experiments

Now let's run some prompt engineering experiments to see how different models respond!

In [ ]:
# Experiment 1: Mathematical Reasoning
math_prompt = "If I have 20 marbles and give away 7, then find 5 more, how many marbles do I have now? Show your reasoning."
platform.compare_models(math_prompt, "🧮 Mathematical Reasoning Test")

In [ ]:
# Experiment 2: Creative Writing
creative_prompt = "Write a short poem about coffee in the morning."
platform.compare_models(creative_prompt, "🎨 Creative Writing Test")

In [ ]:
# Experiment 3: Code Generation
code_prompt = "Write a Python function that reverses a string."
platform.compare_models(code_prompt, "💻 Code Generation Test")

In [ ]:
# Experiment 4: Reasoning and Analysis
reasoning_prompt = "What are the pros and cons of remote work? Give me 3 points for each."
platform.compare_models(reasoning_prompt, "🔍 Reasoning and Analysis Test")

## 5️⃣ Interactive Testing

Use this cell to test your own prompts across all models!

In [ ]:
# Interactive prompt testing
# Change this prompt to test your own ideas!

your_prompt = "Explain artificial intelligence in simple terms that a 12-year-old could understand."

# Run the comparison
platform.compare_models(your_prompt, "🎮 Your Custom Test")

## 6️⃣ Advanced Prompt Engineering Techniques

Let's test some advanced prompting techniques!

In [ ]:
# Chain-of-Thought Prompting
cot_prompt = """
A restaurant bill comes to $120. If I want to tip 18%, how much should I tip?

Let me think step by step:
"""

platform.compare_models(cot_prompt, "🔗 Chain-of-Thought Prompting")

In [ ]:
# Few-Shot Learning Example
few_shot_prompt = """
Classify these movie reviews as positive or negative:

Review: "This movie was amazing! Great acting and plot."
Classification: Positive

Review: "Boring and predictable. Waste of time."
Classification: Negative

Review: "Incredible cinematography and compelling characters."
Classification: Positive

Review: "The story dragged on and the ending was disappointing."
Classification:
"""

platform.compare_models(few_shot_prompt, "📚 Few-Shot Learning")

## 7️⃣ Summary and Next Steps

🎉 **Congratulations!** You now have a complete multi-model prompt engineering environment!

### What you can do next:
- ✏️ Modify the prompts above to test your own ideas
- 🔬 Experiment with different prompting techniques
- 📊 Compare how different models respond to the same prompt
- 📚 Use this setup with your prompt engineering course materials

### Pro Tips:
- Start with simple prompts and gradually make them more complex
- Notice which models excel at different types of tasks
- Try the same prompt with different phrasings
- Use the models' different strengths for different parts of your projects

In [ ]:
# Final verification - let's make sure everything is working
print("🏁 FINAL SETUP VERIFICATION")
print("=" * 30)

# Quick test of each service
services_status = {
    "OpenAI": not platform.test_openai("Hi").startswith("❌"),
    "Llama": not platform.test_llama("Hi").startswith("❌"),
    "Google": not platform.test_google("Hi").startswith("❌")
}

working_count = sum(services_status.values())
total_count = len(services_status)

for service, is_working in services_status.items():
    status = "✅" if is_working else "❌"
    print(f"{status} {service}")

print(f"\n📊 {working_count}/{total_count} services working")

if working_count == total_count:
    print("🎉 PERFECT! All systems operational!")
    print("You have the ultimate prompt engineering setup!")
elif working_count > 0:
    print("✅ GOOD! You can start prompt engineering!")
    print(f"You have {working_count} working AI model(s) to experiment with.")
else:
    print("⚠️  No services working. Please check your .env file and API keys.")

print("\n🚀 Ready for prompt engineering adventures!")